In [2]:
import re
import os
from datetime import datetime

def parse_filename(full_path):
    filename = os.path.basename(full_path)

    # Extracting label from the directory path (assuming it's part of the path)
    path_parts = full_path.split('\\')
    label = path_parts[-2]  # Assuming the label is the second last part of the path

    # Updated regular expression to handle both 'time_point-xx' and a numeric pattern
    pattern = r'(\d{4})_(\d{2})_(\d{2})_(\d{2})h(\d{2})_Plate#(\d+)(?:_([a-zA-Z0-9-]+)|_time_point-(\d+))_AcquisitionBlock(\d+)_pt(\d+)_10x_stitched_Scene(\d+).png'
    match = re.search(pattern, filename)

    if match:
        year, month, day, hour, minute, plate, id_or_time_point, time_point, block, pt, scene = match.groups()
        timestamp = datetime(int(year), int(month), int(day), int(hour), int(minute))

        # Determine if the identifier is a time_point or an ID
        identifier = time_point if time_point else id_or_time_point

        return {
            'timestamp': timestamp,
            'plate': plate,
            'identifier': identifier,  # Can be either an ID or a time_point
            'block': block,
            'pt': pt,
            'scene': scene,
            'label': label
        }
    else:
        return None

In [3]:
from PIL import Image, ImageDraw, ImageFont

def create_gif_with_labels(image_info, output_filename, duration=500):
    images = []

    for label, file_path in image_info:
        # Open the image
        img = Image.open(file_path)
        
        # Optionally resize image if they are not the same size
        # img = img.resize((desired_width, desired_height))

        # Draw the label on the image
        draw = ImageDraw.Draw(img)
        # Use a larger font size with a system TrueType font
        font_size = 40  # Example size, adjust as needed
        font = ImageFont.truetype("arial.ttf", font_size)  # Using Arial as an example
        draw.text((10, 10), label, font=font, fill=(255, 255, 255))

        images.append(img)

    # Save the images as a GIF
    images[0].save(output_filename, save_all=True, append_images=images[1:], loop=0, duration=duration)

In [5]:
import os
import shutil

os.path.split
# Set the directory you want to start from
rootDir = r'E:\New_data_Maxim\images_classified'
outputDir = r'E:\New_data_Maxim\gifs'
file_paths = []
for dirName, subdirList, fileList in os.walk(rootDir):
    label = str(os.path.split(dirName)[-1])
    if label == "images_classified":
        pass
    for fname in fileList:
        if ".png" in fname:
            path = os.path.join(rootDir, label, fname)
            file_paths.append(path)


# Creating a dictionary with paths grouped by similar names except the date, and sorted by date
grouped_sorted_paths = {}
for path in file_paths:
    parsed = parse_filename(path)
    if parsed:
        # Create a key excluding the date
        key = (parsed['plate'], parsed['block'], parsed['pt'], parsed['scene'])
        if key not in grouped_sorted_paths:
            grouped_sorted_paths[key] = []
        grouped_sorted_paths[key].append((parsed['timestamp'], parsed['label'], path))

# Sorting the paths in each group by date
for key in grouped_sorted_paths:
    grouped_sorted_paths[key].sort()

# Using the function in the loop
for key, values in grouped_sorted_paths.items():
    if len(values) < 10:
        # Extract labels and file paths
        # Copy images to the new directory
        image_info = [(value[1], value[2]) for value in values]
        gif_filename = f"{key[-1]}_{len(values)}.gif"  # Name the GIF file after the label in the key

        # Create a directory for each GIF
        gif_dir = os.path.join(outputDir, f"{key[-1]}_images")
        if not os.path.exists(gif_dir):
            os.makedirs(gif_dir)
        for _, image_path in image_info:
            shutil.copy(image_path, gif_dir)
        create_gif_with_labels(image_info, os.path.join(outputDir, gif_filename))

            